In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import pickle
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [ ]:
new_df = pickle.load(open('/content/drive/MyDrive/NLP/new_df1000.pkl', 'rb'))

In [ ]:
# Split the data into training and validation sets
train_df, val_df = train_test_split(new_df, test_size=0.1, random_state=42)

# Tokenization and Formatting
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

In [ ]:
def preprocess_data(data_frame):
    input_texts = data_frame['report'].tolist()
    target_texts = data_frame[['summary_1', 'summary_2', 'summary_3']].apply(lambda x: ' '.join(x), axis=1).tolist()

    # Tokenize input and target texts
    inputs = tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)
    targets = tokenizer(target_texts, return_tensors='pt', padding=True, truncation=True)['input_ids']

    # Include weights as labels for training
    labels = tokenizer(target_texts, return_tensors='pt', padding=True, truncation=True)['input_ids']

    return inputs, targets, labels

In [ ]:
train_input_ids, train_target_ids, train_labels = preprocess_data(train_df)
val_input_ids, val_target_ids, val_labels = preprocess_data(val_df)

In [ ]:
# Define a custom dataset and dataloaders
train_dataset = TensorDataset(train_input_ids['input_ids'], train_target_ids, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = TensorDataset(val_input_ids['input_ids'], val_target_ids, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
# Fine-tune the BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def custom_loss(outputs, labels, weights):
    # Assume outputs and labels are tensors of shape (batch_size, sequence_length, vocab_size)
    # and weights is a tensor of shape (batch_size, sequence_length)

    # Calculate the negative log likelihood loss with weighted sum
    loss = F.nll_loss(F.log_softmax(outputs, dim=-1), labels, reduction='none')
    weighted_loss = (loss * weights.unsqueeze(2)).sum() / weights.sum()

    return weighted_loss

In [ ]:
# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    accumulation_steps = 4  # Accumulate gradients over 4 batches

    for batch_idx, batch in enumerate(train_dataloader):
        inputs, targets, labels = batch
        inputs, targets, labels = inputs.to(device), targets.to(device), labels.to(device)

        outputs = model(inputs, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Accumulate gradients
        if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_dataloader):
            optimizer.step()
            optimizer.zero_grad()



    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_dataloader:
            inputs, targets, labels = batch
            inputs, targets, labels = inputs.to(device), targets.to(device), labels.to(device)

            outputs = model(input_ids=inputs, labels=labels)
            val_loss += outputs.loss.item()

    val_loss /= len(val_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}')

# Save the fine-tuned model if needed
#model.save_pretrained('fine_tuned_model')

Epoch 1/3, Validation Loss: 2.8063
Epoch 2/3, Validation Loss: 2.6128
Epoch 3/3, Validation Loss: 2.6129


In [ ]:
model.save_pretrained('fine_tuned_model')

In [ ]:
import shutil
source_folder = '/content/fine_tuned_model'
destination_folder = '/content/drive/MyDrive/NLP/fine_tuned_model'

shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/NLP/fine_tuned_model'